In [1]:
# Dependencies
import json
import requests
#from config import api_key

In [6]:
# Save config information

url = "http://api.openweathermap.org/data/2.5/air_pollution?"
api_key= "90d059c74bd13bcc4087b66e982c1bf1"
lat= "50"
lon= "50"

# Build query URL
query_url = f"{url}lat={lat}&lon={lon}&appid={api_key}"

In [7]:
# Get weather data
aq_response = requests.get(query_url)
air_json = aq_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {air_json}.")

The weather API responded with: {'coord': {'lon': 50, 'lat': 50}, 'list': [{'main': {'aqi': 1}, 'components': {'co': 158.55, 'no': 0, 'no2': 0.32, 'o3': 57.94, 'so2': 0.09, 'pm2_5': 1.57, 'pm10': 1.67, 'nh3': 0.33}, 'dt': 1626127200}]}.
